In [1]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [2]:
output = []
from sshtunnel import SSHTunnelForwarder
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    printstate = (("doc_id = %s") % (str(doc_id)))
    
    print("Current test: %s" % (str(printstate)))
    
    with SSHTunnelForwarder(('144.214.121.15', 22),
                                ssh_username='ninadt',
                                ssh_password='Ninad123',
                                remote_bind_address=('localhost', 3306),
                                local_bind_address=('localhost', 3300)):
        import UpdateDB as db
        case = db.checkOldGender(doc_id = doc_id)
        
    if case == False:

        print("Running: %12s" % (str(printstate)))

        import StyloGender as Stylo
        (predY, testY) = Stylo.getTestResults(doc_id = doc_id, glove = '../../glove/',
                                              dimensions = 200, chunk_size = 1000, batch_size = 10)
        
        loc = testY

        test_acc = predY[loc]

        test_bin = 0

        if(predY.tolist().index(max(predY)) == testY):
            test_bin = 1
        
        ans = 'N'
        
        if test_bin == 1:
            if testY == 0:
                ans = 'M'
            else:
                ans = 'F'
        else:
            if testY == 1:
                ans = 'M'
            else:
                ans = 'F'
        
        with SSHTunnelForwarder(('144.214.121.15', 22),
                                ssh_username='ninadt',
                                ssh_password='Ninad123',
                                remote_bind_address=('localhost', 3306),
                                local_bind_address=('localhost', 3300)):
            import UpdateDB as db
            case = db.updateresultOldGender(doc_id = doc_id, test_acc = test_acc, test_bin = test_bin, pred = ans)

        output.append([doc_id, test_acc, test_bin])

        del Stylo

        from keras import backend as K
        K.clear_session()

        import time
        time.sleep(10)

        from IPython.display import clear_output
        clear_output()

else:
    print("Skipped: %12s" % (str(printstate)))


Skipped: doc_id = 2318


In [3]:
%tb

No traceback available to show.


In [4]:
output

[[85, 0.99840212, 1],
 [160, 0.93407482, 1],
 [168, 0.99777055, 1],
 [213, 0.91292071, 1],
 [227, 0.88579404, 1],
 [297, 0.86970788, 1],
 [479, 0.99874198, 1],
 [495, 0.99920237, 1],
 [496, 0.99862057, 1],
 [544, 0.93009758, 1],
 [759, 0.9990133, 1],
 [769, 0.89024079, 1],
 [970, 0.99837852, 1],
 [987, 0.49692518, 0],
 [1007, 0.88704962, 1],
 [1095, 0.99719137, 1],
 [1121, 0.99896896, 1],
 [1223, 0.99920589, 1],
 [1444, 0.79767454, 1],
 [1476, 0.99747449, 1],
 [1527, 0.056265883, 0],
 [1727, 0.52917093, 1],
 [1923, 0.72092634, 1],
 [1938, 0.95425314, 1],
 [2010, 0.96851999, 1],
 [2128, 0.9472568, 1],
 [2171, 0.18687579, 0],
 [2175, 0.80857158, 1],
 [2178, 0.98696649, 1],
 [2183, 0.91411275, 1],
 [2259, 0.99923635, 1],
 [2318, 0.54854316, 1]]